You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,  # dont allow delegation to another agent
	verbose=True  # to show agent's inner thoughts
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Unveiling the Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of the importance of Artificial Intelligence in today's technological landscape
- Mention of the rapid advancements and innovations in AI technology

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine Learning and Deep Learning advancements
- Natural Language Processing applications
- AI in healthcare, finance, and autonomous vehicles
- Ethical considerations in AI development

2. Key

I now can give a great answer

Final Answer:
# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become a pivotal force in shaping the technological landscape of today. With rapid advancements and innovations in AI technology, the possibilities seem endless. From enhancing efficiency in various industries to revolutionizing healthcare and finance, AI is at the forefront of driving progress and change.

## Latest Trends in Artificial Intelligence
Machine learning and deep learning have been key drivers of AI progress. These technologies enable machines to learn from data, recognize patterns, and make decisions without explicit programming. The continuous improvements in algorithms and computing power have led to significant breakthroughs in AI capabilities.

Natural Language Processing (NLP) has seen remarkable advancements, allowing machines to understand and generate human language. From virtual assistants to langu

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become a pivotal force in shaping the technological landscape of today. With rapid advancements and innovations in AI technology, the possibilities seem endless. From enhancing efficiency in various industries to revolutionizing healthcare and finance, AI is at the forefront of driving progress and change.

## Latest Trends in Artificial Intelligence
Machine learning and deep learning have been key drivers of AI progress. These technologies enable machines to learn from data, recognize patterns, and make decisions without explicit programming. The continuous improvements in algorithms and computing power have led to significant breakthroughs in AI capabilities.

Natural Language Processing (NLP) has seen remarkable advancements, allowing machines to understand and generate human language. From virtual assistants to language translation tools, NLP is transforming how we interact with technology and each other.

AI is making significant strides in healthcare by improving diagnostics, personalized treatment plans, and drug discovery. In finance, AI is optimizing trading strategies, fraud detection, and customer service. Moreover, autonomous vehicles are leveraging AI for enhanced safety, navigation, and efficiency on the roads.

As AI continues to evolve, ethical considerations have become paramount. Issues such as bias in algorithms, data privacy, and the societal impact of AI implementations need careful attention. It is essential for developers and policymakers to address these ethical challenges to ensure responsible AI deployment.

## Key Players in the AI Industry
The AI industry is populated by key players who are driving innovation and setting the pace for technological advancements.

Google's DeepMind is renowned for its groundbreaking work in AI research, particularly in areas like reinforcement learning and neural networks. Their contributions have paved the way for significant breakthroughs in AI applications.

IBM Watson is a pioneer in cognitive computing, offering AI solutions for businesses across various sectors. From healthcare to finance, IBM Watson's cognitive capabilities have been instrumental in transforming industries.

Amazon Web Services provides a wide range of AI services and tools for developers, enabling them to build sophisticated AI applications. With cloud-based AI solutions, AWS has democratized access to AI technology for businesses of all sizes.

Microsoft Azure offers a comprehensive suite of AI services, including machine learning, cognitive services, and bot frameworks. Their AI solutions empower organizations to harness the power of data and drive intelligent decision-making.

OpenAI is a research organization dedicated to advancing AI in a safe and beneficial manner. Their work focuses on developing AI technologies that are aligned with human values and ethics, promoting transparency and collaboration in AI research.

## Noteworthy News in AI
The field of AI is constantly evolving, with new breakthroughs and discoveries shaping the future of technology. From innovative algorithms to novel applications, AI research is pushing the boundaries of what is possible.

AI is being increasingly integrated into diverse industries, from retail and manufacturing to education and entertainment. The transformative impact of AI applications is evident in optimizing processes, enhancing customer experiences, and driving business growth.

With the rapid expansion of AI technologies, regulatory frameworks and policies are essential to ensure ethical and responsible AI development. Governments and organizations are actively working on establishing guidelines to govern the use of AI and mitigate potential risks.

## Audience Analysis
Our target audience includes tech enthusiasts, business professionals, and students interested in AI. They are eager to explore the latest trends in AI, understand industry applications, and engage in discussions on ethical considerations. By providing insightful content on AI advancements, we aim to address their interests and pain points effectively.

## Call to Action
To stay informed about AI trends and developments, we encourage readers to subscribe to our newsletter for regular updates. Additionally, we invite readers to join the conversation on our social media platforms, where they can engage with like-minded individuals and share their perspectives on AI-related topics.

In conclusion, Artificial Intelligence continues to shape the future of technology with its latest trends, key players, and noteworthy advancements. By staying informed and actively participating in discussions on AI, we can contribute to the responsible development and ethical deployment of AI technologies in society.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).